In [20]:
import platform
if platform.system() in ['Windows', 'Darwin']:
    !pip install selenium > NUL
    !pip install selenium  > NUL
    !pip install webdriver_manager  > NUL
    !pip install docker  > NUL
    !pip install requests  > NUL
elif platform.system() == 'Linux':
    !pip install selenium  > /dev/null
    !pip install webdriver_manager  > /dev/null
    !pip install docker  > /dev/null
    !pip install requests  > /dev/null

^C
ERROR: Operation cancelled by user


##  Get the container ID

In [ ]:
import subprocess

# Get the first container ID
#get_cont_id_cmd = "docker ps -q | head -n 1"
get_cont_id_cmd = "/usr/local/bin/docker ps -q | head -n 1"
cont_id = subprocess.check_output(get_cont_id_cmd, shell=True).decode('utf-8').strip()

print("Container ID:", cont_id)

Container ID: 


Cannot connect to the Docker daemon at unix:///Users/zhangyiqin/.docker/run/docker.sock. Is the docker daemon running?


## Get the Jupyter lab token

In [ ]:
get_token_cmd = f"docker exec {cont_id} jupyter lab list 2>&1 | grep -o 'token=[^ ]*' | cut -d= -f2"
token = subprocess.check_output(get_token_cmd, shell=True).decode('utf-8').strip()

print("Jupyter Lab Token:", token)

Jupyter Lab Token: 


##  Run the rise_test.py 

In [ ]:
# Run the rise_test.py with the token
if token:
    run_script_cmd = f"python3 rise_test.py {token}"
    script_output = subprocess.check_output(run_script_cmd, shell=True).decode('utf-8')
    print("Script Output:\n", script_output)    
else:
    print("Token not found. Check if Jupyter Lab is running in the container.")

Token not found. Check if Jupyter Lab is running in the container.


## Test the RISE

In [2]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import platform
import subprocess

In [3]:
# replace token with your token
token = "bd34fbdf30ca0c9ff4a1b18c8ac96fa30e023bf1e151318a"

In [4]:
def setup_driver():
    chrome_options = Options()
    if platform.system() in ['Windows', 'Darwin']:
        chrome_options.add_experimental_option('detach', True)
    elif platform.system() == 'Linux':
        chrome_options.add_argument('--headless')
    else:
        raise Exception("Unknown OS")
    return webdriver.Chrome(service=ChromiumService(ChromeDriverManager().install()), options=chrome_options)

In [5]:
base_url = "http://localhost:8888"  # Base URL of your JupyterLab server
image_path = "/lab/images/sample-image.jpg"  # Extracted relative path
full_image_url = base_url + image_path
image_url="https://raw.githubusercontent.com/YiqinZhang/ope-project/e9ea65fa2317b3c39d70984875ce2885dde8e599/content/images/sample-image.jpg"

In [6]:
def create_jupyter_notebook_from_github(github_url, local_file_name):
    try:
        response = requests.get(github_url)
        response.raise_for_status()  # Raise an error for bad status codes
        notebook_content = response.json()
        with open(local_file_name, 'w', encoding='utf-8') as notebook_file:
            json.dump(notebook_content, notebook_file)
        print(f"Notebook created: {local_file_name}")
    except requests.RequestException as e:
        print(f"Failed to download the notebook: {e}")

In [7]:
import os
def download_image(image_url, local_img_path):
    response = requests.get(image_url, stream=True)

    if response.status_code == 200:
        os.makedirs(os.path.dirname(local_img_path), exist_ok=True)
        with open(local_img_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)
        print(f"Image downloaded successfully: {local_img_path}")
    else:
        print(f"Failed to download the image: {image_url}")

image_url = 'https://raw.githubusercontent.com/YiqinZhang/ope-project/e9ea65fa2317b3c39d70984875ce2885dde8e599/content/images/sample-image.jpg'
download_image(image_url, 'images/sample-image.jpg')


Image downloaded successfully: images/sample-image.jpg


In [8]:
notebook_github_url = "https://raw.githubusercontent.com/OPEFFORT/ope-project/main/content/test_book/02_slides_template/layout_example.ipynb"
local_notebook_name = "downloaded_notebook.ipynb"
create_jupyter_notebook_from_github(notebook_github_url, local_notebook_name)

Notebook created: downloaded_notebook.ipynb


In [9]:
def open_file_in_jupyterlab(url, file_name, driver):
    try:
        driver.get(url)
        # Wait for the file browser to be available
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".jp-DirListing-content"))
        )

        # Find the file in the file browser
        file_element = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, f"//span[contains(text(), '{file_name}') and @class='jp-DirListing-itemText']"))
        )

        # Double click on the file to open it
        driver.execute_script("""
            var evt = new MouseEvent('dblclick', {
                bubbles: true,
                cancelable: true,
                view: window
            });
            arguments[0].dispatchEvent(evt);
        """, file_element)

        print(f"File '{file_name}' opened successfully.")
    except TimeoutException:
        print(f"Failed to open file '{file_name}'. File not found or JupyterLab did not load properly.")

In [10]:
def take_screenshot(driver, filename):
    try:
        driver.save_screenshot(filename)
        print(f"Saved screenshot: {filename}")
    except Exception as e:
        print(f"Error taking screenshot: {e}")


In [11]:
def find_element_and_click(driver, xpath):
    try:
        element = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        # element = driver.find_element(By.XPATH, xpath)
        element.click()
        return element
    except Exception as e:
        print(f"Error finding/clicking element: {e}")


## Test1: Refactor the Previous Test for Suitability with the Launcher

In [12]:
def test_rise_extension(token):
    driver = setup_driver()
    url = f"http://127.0.0.1:8888/lab?token={token}"
    try:
        driver.get(url)
        driver.implicitly_wait(45)
        #used to return to base screen for next test
        main_window_handle = driver.current_window_handle 

        # Main actions and interactions
        take_screenshot(driver, 'main_screenshot.png')

        find_element_and_click(driver, '//div[@data-category="Notebook"]')
        take_screenshot(driver, 'new_nb_screenshot.png')

        find_element_and_click(driver, '//button[@data-command="RISE:preview"]')
        take_screenshot(driver, 'rise_screenshot.png')

        toolbar = WebDriverWait(driver, 20).until(EC.presence_of_element_located
                 ((By.XPATH, "//div[@class='lm-Widget jp-Toolbar' and @role='navigation']")))
        if toolbar:
            fullscreen_button = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, ".//button[@title='Open the slideshow in full screen']"))
            )
            fullscreen_button.click()
            is_fullscreen = driver.execute_script("return document.fullscreenElement !== null")
            print("Fullscreen button is functioning: ", is_fullscreen)
        else:
            print("Toolbar not found")
        print("RISE Extension is working!")
    except Exception as e:
        print(f"An error occurred during test: {e}")
    finally:
        # Clean up
        driver.switch_to.window(main_window_handle)
        driver.refresh()
        WebDriverWait(driver, 90) # wait for page to load
        driver.quit()

In [13]:
test_rise_extension(token)

Saved screenshot: main_screenshot.png
Saved screenshot: new_nb_screenshot.png
Saved screenshot: rise_screenshot.png
Fullscreen button is functioning:  False
RISE Extension is working!


In [14]:
subprocess.run(["python", "screenshots_diff.py"])

Images to compare: ['new_nb_screenshot.png', 'rise_screenshot.png', 'main_screenshot.png']
new_nb_screenshot.png are identical.
rise_screenshot.png are identical.
ERROR: main_screenshot.png is blank.


CompletedProcess(args=['python', 'screenshots_diff.py'], returncode=0)

## Test 2: Downloaded Testbook Slides for the RISE Test

In [15]:
def test_rise(token, notebook_github_url, local_notebook_name):
    driver = setup_driver()
    create_jupyter_notebook_from_github(notebook_github_url, local_notebook_name)
    open_file_in_jupyterlab(f"http://127.0.0.1:8888/lab?token={token}", local_notebook_name,driver)

    try:
        WebDriverWait(driver, 45)
        take_screenshot(driver, 'main_screenshot.png')
        main_window_handle = driver.current_window_handle 
        # Correct the XPath and check if the "Launcher" tab is present
        try:
            # launcher_element = driver.find_element(By.XPATH, '//div[@data-category="Notebook"]')
            launcher_element = driver.find_element(By.XPATH, 
            "//div[contains(@class, 'lm-TabBar-tabLabel') and contains(@class, 'p-TabBar-tabLabel') and text()='Launcher']")
            is_launcher_page = True
        except NoSuchElementException:
            is_launcher_page = False 
        
        if is_launcher_page:
            print("On the Launcher page")
            find_element_and_click(driver, '//div[@data-category="Notebook"]')
        else:
            print("Not on the Launcher page, trying to open the notebook directly")
            open_file_in_jupyterlab(f"http://127.0.0.1:8888/lab?token={token}", local_notebook_name,driver)
     
        take_screenshot(driver, 'new_nb_screenshot.png')
        
        find_element_and_click(driver, '//button[@data-command="RISE:preview"]')        
        take_screenshot(driver, 'rise_screenshot.png')

        fullscreen_button = find_element_and_click(driver, ".//button[@title='Open the slideshow in full screen']")
        is_fullscreen = driver.execute_script("return document.fullscreenElement !== null")
        print("Fullscreen button is functioning: ", is_fullscreen)

    finally:
        # Clean up  
        driver.switch_to.window(main_window_handle)
        driver.refresh()
        WebDriverWait(driver, 90)
        driver.quit()

In [16]:
test_rise(token, notebook_github_url, local_notebook_name)

Notebook created: downloaded_notebook.ipynb
Failed to open file 'downloaded_notebook.ipynb'. File not found or JupyterLab did not load properly.
Saved screenshot: main_screenshot.png
Not on the Launcher page, trying to open the notebook directly
Failed to open file 'downloaded_notebook.ipynb'. File not found or JupyterLab did not load properly.
Saved screenshot: new_nb_screenshot.png
Error finding/clicking element: Message: 
Stacktrace:
0   chromedriver                        0x000000010e06dc48 chromedriver + 4852808
1   chromedriver                        0x000000010e0651b3 chromedriver + 4817331
2   chromedriver                        0x000000010dc317bd chromedriver + 411581
3   chromedriver                        0x000000010dc81e30 chromedriver + 740912
4   chromedriver                        0x000000010dc81fe1 chromedriver + 741345
5   chromedriver                        0x000000010dccd864 chromedriver + 1050724
6   chromedriver                        0x000000010dcabc2d chromedrive

In [17]:
subprocess.run(["python", "screenshots_diff.py"])

Images to compare: ['new_nb_screenshot.png', 'rise_screenshot.png', 'main_screenshot.png']
new_nb_screenshot.png are identical.
rise_screenshot.png are identical.
main_screenshot.png are identical.


CompletedProcess(args=['python', 'screenshots_diff.py'], returncode=0)

## Test 3: Create slides and test RISE

In [18]:
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell

# Define the cells with their content and metadata
cells = [
    new_markdown_cell("# Example Slidedeck", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("<hr>", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("**0) Blank**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("**BODY**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("We will try to illustrate single-column slides like this one.", metadata={"slideshow": {"slide_type": "slide"}}),
    new_markdown_cell("<hr>", metadata={"slideshow": {"slide_type": "skip"}}),  
    new_markdown_cell(source="**1) Image Only**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell(source="**BODY**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell(source='<img src="../images/sample-image.jpg" width = "100%">', 
                      metadata={"slideshow": {"slide_type": "slide"}}),
    new_markdown_cell(source="**NOTE**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell(source="- Using % of slide to scale the image", 
                      metadata={"slideshow": {"slide_type": "notes"}}),
    new_markdown_cell(source="- - - ", metadata={"slideshow": {"slide_type": "skip"}}),  
    new_markdown_cell("**7)Title Slide**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("# Title", metadata={"slideshow": {"slide_type": "slide"}}),
    new_markdown_cell("**NOTE**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("- a Note that will display in the notes view", metadata={"slideshow": {"slide_type": "notes"}}),
    new_markdown_cell("This is the end of the document.", metadata={"slideshow": {"slide_type": "skip"}, "tags": ["remove-cell"]}),
]

In [24]:
def test_rise_slide(token):
    driver = setup_driver()
    url = f"http://127.0.0.1:8888/lab?token={token}"
    try:
        driver.get(url)
        WebDriverWait(driver, 45)
        main_window_handle = driver.current_window_handle 
        # Main actions and interactions       
        take_screenshot(driver, 'main_screenshot.png')
        # Check if on the Launcher page
        try:
            driver.find_element(By.XPATH, "//div[contains(@class, 'lm-TabBar-tabLabel') and contains(@class, 'p-TabBar-tabLabel') and text()='Launcher']")
            print("On the Launcher page")
            find_element_and_click(driver, '//div[@data-category="Notebook"]')
        except NoSuchElementException:
            print("Not on the Launcher page")
        # Create a new notebook object
        nb = new_notebook()
        # Add cells to the notebook
        nb['cells'] = cells
        # Write the notebook to a file
        with open('example_notebook.ipynb', 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)
        print("Notebook created successfully.")
        take_screenshot(driver, 'new_nb_screenshot.png')

        find_element_and_click(driver, '//button[@data-command="RISE:preview"]')
        take_screenshot(driver, 'rise_screenshot.png')

        fullscreen_button = find_element_and_click(driver, ".//button[@title='Open the slideshow in full screen']")
        is_fullscreen = driver.execute_script("return document.fullscreenElement !== null")
        print("Fullscreen button is functioning: ", is_fullscreen)
    except Exception as e:
        print(f"An error occurred during test: {e}")
    finally:
        # Clean up
        driver.switch_to.window(main_window_handle)
        driver.refresh()
        WebDriverWait(driver, 90)
        driver.quit()

In [25]:
test_rise_slide(token)

Saved screenshot: main_screenshot.png
Not on the Launcher page
Notebook created successfully.
Saved screenshot: new_nb_screenshot.png
Error finding/clicking element: Message: 
Stacktrace:
0   chromedriver                        0x000000010df32c48 chromedriver + 4852808
1   chromedriver                        0x000000010df2a1b3 chromedriver + 4817331
2   chromedriver                        0x000000010daf67bd chromedriver + 411581
3   chromedriver                        0x000000010db46e30 chromedriver + 740912
4   chromedriver                        0x000000010db46fe1 chromedriver + 741345
5   chromedriver                        0x000000010db92864 chromedriver + 1050724
6   chromedriver                        0x000000010db70c2d chromedriver + 912429
7   chromedriver                        0x000000010db8fae2 chromedriver + 1039074
8   chromedriver                        0x000000010db709a3 chromedriver + 911779
9   chromedriver                        0x000000010db38103 chromedriver + 68019

In [21]:
subprocess.run(["python", "screenshots_diff.py"])

Images to compare: ['new_nb_screenshot.png', 'rise_screenshot.png', 'main_screenshot.png']
ERROR: new_nb_screenshot.png is blank.
rise_screenshot.png are identical.
ERROR: main_screenshot.png is blank.


CompletedProcess(args=['python', 'screenshots_diff.py'], returncode=0)

## Test 4: Test terminal 

In [22]:
def test_terminal(token):
    driver = setup_driver()
    url = f"http://127.0.0.1:8888/lab?token={token}"

    try:
        driver.get(url)
        print("Opened JupyterLab.")
        #used to return to base screen for next test
        main_window_handle = driver.current_window_handle 
        try:
            # Try to find the Launcher button
            launcher_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, '//button[@data-command="launcher:create"]'))
            )
            launcher_button.click()
            print("Opened the Launcher.")
        except:
            print("Launcher button not found, skipping to Terminal.")

        # Click on the Terminal icon in the Launcher
        terminal_icon = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@title="Start a new terminal session"]'))
        )
        terminal_icon.click()
        print("Opened Terminal.")
        terminal_input = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'jp-Terminal')]//textarea"))
        )
        print("Terminal is ready for input.")    
        # Find terminal input and execute a command
        terminal_input.send_keys('echo Hello World' + Keys.ENTER)
        print("Command sent to terminal.")   
        # Find terminal output and print it
        output_xpath = "//div[contains(@class, 'jp-Terminal-body')]"
        try:
            output = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, output_xpath))
            )
            print("Captured terminal output:" + output.text)
        except Exception as e:
            print(f"An error occurred during test: {e}")
        print("Terminal test successful.")
        # Close the terminal
        close_terminal_xpath = "//div[contains(@class, 'jp-icon-hover') and contains(@class, 'lm-TabBar-tabCloseIcon')]"
        try:
            close_terminal_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, close_terminal_xpath))
            )
            close_terminal_button.click()
            print("Closed Terminal.")
        except Exception as e:
            print("Error closing terminal: ", e)
    except Exception as e:
        print(f"An error occurred during test: {e}")
    finally:
        driver.switch_to.window(main_window_handle)
        driver.refresh()
        driver.quit()

In [23]:
test_terminal(token)

Opened JupyterLab.
Opened the Launcher.
Opened Terminal.
Terminal is ready for input.
Command sent to terminal.
Captured terminal output:
Terminal test successful.
Closed Terminal.
